In [1]:
%matplotlib inline
%matplotlib notebook

import itertools
import numpy as np
import os
import random
import sys
import psutil
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation
import math
import matplotlib as mp
plt.switch_backend('agg')

if "./lib" not in sys.path:
  sys.path.append("./lib")

import plotting
from collections import deque, namedtuple
from readingFileEfficiently import *
import VOC2012_npz_files_writter
from DNN import *
from Agent import ObjLocaliser

/home/mohammad/Tensorflow/local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [2]:
# Path to .npz files
destination = "../data/"

# Checks if the input files are ready to be read 

if not (os.path.isfile(destination+"test_input.npz") or os.path.isfile(destination+"test_target.npz")):
    print "Please first prepare .npz files. You can not proceed!"
else:
    print("Records are already prepared!!!")

Records are already prepared!!!


In [3]:
# Name of the model you wish to load and test
model_name = "allObj"


tf.reset_default_graph()

# Where the checkpoints and graphs are saved
experiment_dir = os.path.abspath("/home/mohammad/active localization/trained_on_cluster_final/{}".format(model_name))

# Create a glboal step variable
global_step = tf.Variable(0, name='global_step', trainable=False)
    
# Create estimators
q_estimator = Estimator(scope="q_estimator", summaries_dir=experiment_dir)

# State processor
state_processor = StateProcessor()

# Category of objects you want to test your model on.
category = ['cat', 'dog', 'car', 'chair']


#fig = plt.figure()
ims = []

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer()) # Old API: tf.initialize_all_variables()

    # Create directories for checkpoints and summaries
    checkpoint_dir = os.path.join(experiment_dir, "checkpoints")
    checkpoint_path = os.path.join(checkpoint_dir, "model")

    saver = tf.train.Saver()
    # Load a previous checkpoint if we find one
    latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    if latest_checkpoint:
        print("Loading model checkpoint {}...\n".format(latest_checkpoint))
        saver.restore(sess, latest_checkpoint)



    # The policy we're following
    policy = make_epsilon_greedy_policy(q_estimator, len(VALID_ACTIONS))
    
    precisions = []
    
    for indx,tmp in enumerate(extractData(category, "test", 32)):

        img = tmp[0]
        target = tmp[1]

        im2 = Image.frombytes("RGB",(img['image_width'],img['image_height']),img['image'])
        env = ObjLocaliser(np.array(im2),target)
        print "New image is being loaded: {}".format(img['image_filename'])
        succ_proposals = 0
        for step in range(15):
            
            
            #ims = []
            #fig = plt.figure()

            # Reset the environment
            env.Reset(np.array(im2))
            state = env.wrapping()
            state = state_processor.process(sess, state)
            state = np.stack([state] * 4, axis=2)
            action = 0
            t = 0
            

            # One step in the environment
            while (action != 10) and (t < 50):

                # Return a list of probabilities for actions with 100% greedy policy
                action_probs = policy(sess, state, 0.2)
                #print action_probs
                action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
                t += 1

                # Taking the action
                reward = env.takingActions(VALID_ACTIONS[action])
                if reward == 3:
                    succ_proposals += 1
                # Reading the next state
                next_state = env.wrapping()

                # For visulization porpuses
                #imgplot = plt.imshow(env.my_draw())
                #ims.append([imgplot])

                # Processing the next state
                next_state = state_processor.process(sess, next_state)
                next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)
            print "number of actions for step {} is: {}".format(step, t)
        precisions.append(succ_proposals/50.0)
        print "Precision for image {} is: {}".format(img['image_filename'], precisions[-1])


        
#ani = animation.ArtistAnimation(fig, ims, interval=1000, blit=True, repeat_delay=1000)
#ani.save('./graphs/successful.mp4')

Instructions for updating:
Please switch to tf.train.get_global_step


/home/mohammad/Tensorflow/local/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Loading model checkpoint /home/mohammad/active localization/trained_on_cluster_final/allObj/checkpoints/model...

INFO:tensorflow:Restoring parameters from /home/mohammad/active localization/trained_on_cluster_final/allObj/checkpoints/model
input file is loading...
New image is being loaded: 2008_004402.jpg
number of actions for step 0 is: 4
number of actions for step 1 is: 35
number of actions for step 2 is: 14
number of actions for step 3 is: 36
number of actions for step 4 is: 30
Precision for image 2008_004402.jpg is: 0.0
New image is being loaded: 2010_000782.jpg
number of actions for step 0 is: 20
number of actions for step 1 is: 50
number of actions for step 2 is: 50
number of actions for step 3 is: 1
number of actions for step 4 is: 37
Precision for image 2010_000782.jpg is: 0.0
New image is being loaded: 2008_003252.jpg
number of actions for step 0 is: 45
number of actions for step 1 is: 18
number of actions for step 2 is: 33
number of actions for step 3 is: 50
number of actio

number of actions for step 3 is: 50
number of actions for step 4 is: 16
Precision for image 2009_001139.jpg is: 0.0
New image is being loaded: 2007_008415.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 50
number of actions for step 2 is: 50
number of actions for step 3 is: 30
number of actions for step 4 is: 15
Precision for image 2007_008415.jpg is: 0.0
New image is being loaded: 2008_008578.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 31
number of actions for step 2 is: 50
number of actions for step 3 is: 39
number of actions for step 4 is: 35
Precision for image 2008_008578.jpg is: 0.0
New image is being loaded: 2011_002177.jpg
number of actions for step 0 is: 34
number of actions for step 1 is: 50
number of actions for step 2 is: 50
number of actions for step 3 is: 50
number of actions for step 4 is: 8
Precision for image 2011_002177.jpg is: 0.02
New image is being loaded: 2010_005984.jpg
number of actions for step 0 is: 50
numb

number of actions for step 3 is: 50
number of actions for step 4 is: 24
Precision for image 2010_004813.jpg is: 0.04
New image is being loaded: 2008_002199.jpg
number of actions for step 0 is: 44
number of actions for step 1 is: 50
number of actions for step 2 is: 50
number of actions for step 3 is: 40
number of actions for step 4 is: 50
Precision for image 2008_002199.jpg is: 0.0
New image is being loaded: 2007_008260.jpg
number of actions for step 0 is: 3
number of actions for step 1 is: 49
number of actions for step 2 is: 26
number of actions for step 3 is: 42
number of actions for step 4 is: 27
Precision for image 2007_008260.jpg is: 0.0
New image is being loaded: 2010_003060.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 33
number of actions for step 2 is: 19
number of actions for step 3 is: 10
number of actions for step 4 is: 48
Precision for image 2010_003060.jpg is: 0.0
New image is being loaded: 2010_003551.jpg
number of actions for step 0 is: 24
numb

...

number of actions for step 0 is: 13
number of actions for step 1 is: 50
number of actions for step 2 is: 16
number of actions for step 3 is: 50
number of actions for step 4 is: 5
Precision for image 2010_002269.jpg is: 0.0
New image is being loaded: 2010_002844.jpg
number of actions for step 0 is: 34
number of actions for step 1 is: 50
number of actions for step 2 is: 50
number of actions for step 3 is: 50
number of actions for step 4 is: 50
Precision for image 2010_002844.jpg is: 0.0
New image is being loaded: 2010_003385.jpg
number of actions for step 0 is: 22
number of actions for step 1 is: 45
number of actions for step 2 is: 7
number of actions for step 3 is: 50
number of actions for step 4 is: 7
Precision for image 2010_003385.jpg is: 0.04
New image is being loaded: 2011_002724.jpg
number of actions for step 0 is: 1
number of actions for step 1 is: 17
number of actions for step 2 is: 46
number of actions for step 3 is: 50
number of actions for step 4 is: 50
Precision for image 20

number of actions for step 0 is: 6
number of actions for step 1 is: 7
number of actions for step 2 is: 50
number of actions for step 3 is: 50
number of actions for step 4 is: 37
Precision for image 2010_005903.jpg is: 0.04
New image is being loaded: 2010_004879.jpg
number of actions for step 0 is: 4
number of actions for step 1 is: 26
number of actions for step 2 is: 4
number of actions for step 3 is: 2
number of actions for step 4 is: 19
Precision for image 2010_004879.jpg is: 0.0
New image is being loaded: 2008_003719.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 40
number of actions for step 2 is: 19
number of actions for step 3 is: 43
number of actions for step 4 is: 44
Precision for image 2008_003719.jpg is: 0.0
New image is being loaded: 2011_003259.jpg
number of actions for step 0 is: 5
number of actions for step 1 is: 2
number of actions for step 2 is: 9
number of actions for step 3 is: 22
number of actions for step 4 is: 50
Precision for image 2011_0

number of actions for step 0 is: 27
number of actions for step 1 is: 50
number of actions for step 2 is: 16
number of actions for step 3 is: 21
number of actions for step 4 is: 50
Precision for image 2011_001944.jpg is: 0.0
New image is being loaded: 2009_004855.jpg
number of actions for step 0 is: 15
number of actions for step 1 is: 16
number of actions for step 2 is: 50
number of actions for step 3 is: 50
number of actions for step 4 is: 22
Precision for image 2009_004855.jpg is: 0.0
New image is being loaded: 2010_000175.jpg
number of actions for step 0 is: 13
number of actions for step 1 is: 50
number of actions for step 2 is: 27
number of actions for step 3 is: 50
number of actions for step 4 is: 50
Precision for image 2010_000175.jpg is: 0.0
New image is being loaded: 2008_004559.jpg
number of actions for step 0 is: 33
number of actions for step 1 is: 33
number of actions for step 2 is: 24
number of actions for step 3 is: 31
number of actions for step 4 is: 26
Precision for image

number of actions for step 2 is: 31
number of actions for step 3 is: 37
number of actions for step 4 is: 50
Precision for image 2010_001656.jpg is: 0.04
New image is being loaded: 2009_000203.jpg
number of actions for step 0 is: 18
number of actions for step 1 is: 50
number of actions for step 2 is: 41
number of actions for step 3 is: 13
number of actions for step 4 is: 50
Precision for image 2009_000203.jpg is: 0.0
New image is being loaded: 2009_000303.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 3
number of actions for step 2 is: 26
number of actions for step 3 is: 50
number of actions for step 4 is: 7
Precision for image 2009_000303.jpg is: 0.04
New image is being loaded: 2010_001386.jpg
number of actions for step 0 is: 9
number of actions for step 1 is: 50
number of actions for step 2 is: 29
number of actions for step 3 is: 50
number of actions for step 4 is: 38
Precision for image 2010_001386.jpg is: 0.0
New image is being loaded: 2008_006999.jpg
numbe

number of actions for step 2 is: 39
number of actions for step 3 is: 50
number of actions for step 4 is: 12
Precision for image 2010_000043.jpg is: 0.0
New image is being loaded: 2008_007344.jpg
number of actions for step 0 is: 33
number of actions for step 1 is: 50
number of actions for step 2 is: 46
number of actions for step 3 is: 50
number of actions for step 4 is: 50
Precision for image 2008_007344.jpg is: 0.0
New image is being loaded: 2008_003576.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 9
number of actions for step 2 is: 28
number of actions for step 3 is: 50
number of actions for step 4 is: 26
Precision for image 2008_003576.jpg is: 0.0
New image is being loaded: 2008_002623.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 50
number of actions for step 2 is: 50
number of actions for step 3 is: 6
number of actions for step 4 is: 50
Precision for image 2008_002623.jpg is: 0.0
New image is being loaded: 2008_006587.jpg
number

In [4]:
len(precisions)

977

In [5]:
np.mean(precisions)

0.006386898669396111

In [6]:
b = [1 if i > 0 else 0 for i in precisions ]

In [7]:
np.mean(b)

0.1780962128966223

"cats+dog+car+chair" + "step=5" + "epsilon=1" : 0.3254861821903787